# Решение тестового задания от Bewise.ai

**Описание задания**

Написать скрипт для парсинга диалогов из файла `test_data.csv`, который должен решать следующие задачи:
* извлекать реплики с приветствием, в которых менеджер поздоровался;
* извлекать реплики, где менеджер представил себя;
* извлекать имя менеджера;
* извлекать название компании;
* извлекать реплики, где менеджер попрощался;
* проверять требование к менеджеру, заключающее в том, чтобы в каждом диалоге он здоровался и прощался с клиентом.

## Загрузка и анализ данных

In [1]:
# импорт необходимых библиотек
import pandas as pd
import numpy as np
from ast import literal_eval

import spacy
import torch
from glob import glob

from sklearn.metrics import f1_score

In [2]:
# загрузка и чтение данных
df = pd.read_csv('./datasets/nlp/test_data.csv')

In [3]:
# оценка размера загруженного датасета
df.shape

(480, 4)

In [4]:
# просмотр первых строк данных
df.head(5)

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,manager,Ага


In [5]:
# просмотр сводной информации о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [6]:
# проверка наличия пропущенных значений
df.isna().sum()

dlg_id    0
line_n    0
role      0
text      0
dtype: int64

In [7]:
# проверка наличия дубликатов
df.duplicated().sum()

0

### Выводы по анализу данных

> * Исходные данные представлены в виде таблицы, содержащей `480` записей в виде таблицы со следующими столбцами:
>     * `dlg_id` - id (порядковый номер) диалога
>     * `line_n` - порядковый номер строки с репликой со сквозной нумерацией
>     * `role`   - сторона, принимающая участие в диалоге (`client` или `manager`)
>     * `text`   - текст реплики в диалоге
> * Всего в полученных данных обнаружено `6` уникальных диалогов.
> * Дубликатов и пропущенных значений в исходном наборе данных нет.
> * Судя по отсутствию текстовой разметки, данные получены в результате работы `STT` (`speech-to-text`) модели.
> * Задача, в целом, сводится к выделению необходимых именованных сущностей (`NER`) из реплик.
> * В качестве метрики будем использовать *F1*-меру как гармоническое среднее между точностью (*precision*) и полнотой (*recall*), вычисляемую по формуле:
$$
F = 2 ∙ \frac{Recall × Precision}{Recall + Precision}
$$
>
>
> * Интерпретировать метрики *precision* и *recall* в данной задаче можно следующим образом:
>     - точность (*precision*) будет тем выше, чем меньше ложноположительных сущностей NER (*FP*) определит модель, т.е. чем меньше неименованных сущностей будет установлено как именованные;
>     - полнота (*recall*) будет тем выше, чем меньше ложноотрицательных сущностей NER (*FN*) установит модель, т.е. чем меньше она их пропустит.

## Использование моделей для решения NER задачи

Так как получен датасет небольшого размера, поставленную задачу можно было решить с помощью `rule-based` подхода, но он плохо масштабируется при увеличении размеров анализируемых данных, так как количество правил для выделения именованных сущностей будет постоянно расти вместе с трудозатратами на их обновление.

Поэтому, от него сразу было решено отказаться и для выделения именованных сущностей `NER` использовать библиотеку `spaCy` в основе которой лежит применение свёрточных нейронных сетей для обработки текстов.

За основу была взята предобученная стандартная модель `ru_core_news_lg`.

Для решения поставленной задачи модель должна выделять три базовых типа именованных сущностей из текста:
* `PERSON` - имя менеджера;
* `COMPANY` - название компании;
* `EVENT` - событие (приветствие или прощание).

По качеству выделения данных сущностей и будет проводится оценка модели.

### Разметка данных для последующей оценки качества моделей

Чтобы иметь возможность оценить по *F1*-мере качество работы различных моделей, была произведена ручная разметка датасета с использованием библиотеки `spacy_annotator` с помощью кода, пример которого приведён ниже в закомментированном виде.

In [8]:
#import spacy_annotator as spa

#nlp = spacy.load('ru_core_news_lg')
#annotator = spa.Annotator(labels=['COMPANY', 'PERSON', 'EVENT'], model=nlp)
#df_labels = annotator.annotate(df=df[['text']], col_text='text', shuffle=False)

#df_labels.to_csv('./Documents/NER2/text_labels_without_indexes.csv', index=False)

Размеченные данные сохранены в файл `text_labels_without_indexes.csv` и выглядят следующим образом.

In [9]:
# просмотр первых строк размеченных данных
df_marked = pd.read_csv('./Documents/NER2/text_labels_without_indexes.csv')
df_marked.head()

,text,annotations
0,Алло,"('Алло', {'entities': []})"
1,Алло здравствуйте,"('Алло здравствуйте', {'entities': [(5, 17, 'E..."
2,Добрый день,"('Добрый день', {'entities': [(0, 11, 'EVENT')]})"
3,Меня зовут ангелина компания диджитал бизнес з...,('Меня зовут ангелина компания диджитал бизнес...
4,Ага,"('Ага', {'entities': []})"


Для удобства последующего расчёта выбранной метрики качества моделей добавим соотвествующие метки для трёх классов сущностей (`event`, `company`, `person`) к исходному датасету.

In [10]:
# конвертация данных из строкового представления в список
marked_list = df_marked['annotations'].apply(lambda x: literal_eval(str(x))).values

In [11]:
# добавление меток классов именованных сущностей в исходный датасет
j = 0
for entities in marked_list:
    if len(entities[1]['entities']):
        for ent in entities[1]['entities']:
            if ent[2]=='EVENT':
                df.loc[j, 'event'] = 1
            elif ent[2]=='COMPANY':
                df.loc[j, 'company'] = 1
            elif ent[2]=='PERSON':
                df.loc[j, 'person'] = 1
    j+=1
    
df[['event', 'company', 'person']] = df[['event', 'company', 'person']].fillna(0)

In [12]:
# просмотр полученного результата
df.head()

,dlg_id,line_n,role,text,event,company,person
0,0,0,manager,Алло,0.0,0.0,0.0
1,0,1,client,Алло здравствуйте,1.0,0.0,0.0
2,0,2,manager,Добрый день,1.0,0.0,0.0
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,0.0,1.0,1.0
4,0,4,manager,Ага,0.0,0.0,0.0


### Baseline решение на основе библиотеки spaCy

В качестве *baseline* решения будем использовать результаты работы предобученной модели для русского языка из библиотеки `spaCy`.

In [13]:
# загрузка стандартной модели
nlp = spacy.load('ru_core_news_lg')

In [14]:
# получение именованных сущностей и добавление соответствующих меток в датасет для последующего расчёта метрик
line_n = []
for i, s in enumerate(df['text'].values):
    doc = nlp(s)
    if doc.ents:
        for ent in doc.ents:
            print(i, ent.text, ent.label_)
            if ent.label_=='EVT':
                df.loc[i, 'event_mod'] = 1
            elif ent.label_=='ORG':
                df.loc[i, 'company_mod'] = 1
            elif ent.label_=='PER':
                df.loc[i, 'person_mod'] = 1
            line_n.append(i)
            
df[['company_mod', 'person_mod']] = df[['company_mod', 'person_mod']].fillna(0)

3 ангелина PER
111 ангелина PER
159 денис дэ е эн PER
167 ангелина PER
251 максим PER
251 китобизнес PER
253 анастасия PER
282 амо ORG
285 амо си эр эм ORG
319 айдар PER
341 дима PER
348 ла ла ла PER
358 анастасия PER
410 амо ORG
438 вячеслав PER
444 максим PER


In [15]:
# вычисление метрик для baseline модели
df_res = pd.DataFrame([f1_score(df['person'], df['person_mod']), f1_score(df['company'], df['company_mod']), 0],
             index=['PER', 'ORG', 'MEAN'], columns=['F1'])
df_res.loc['MEAN', 'F1'] = df_res.iloc[:2, 0].mean()
df_res

,F1
PER,0.750
ORG,0.000
MEAN,0.375


> * *Baseline* решение на основе предобученной модели для библиотеки `spaCy` позволило достичь среднего значения между тремя видами сущностей по *F1*-мере, равного `0.38`.
> * Модель достаточно хорошо справляется с выделением именованных сущностей в виде имён (`PER`) со значением метрики *F1* в данной категории, равным `0.75` и совсем не определяет сущности в виде названий компаний (`ORG`) и событий в виде приветствий и прощаний (`EVT`).

### Добавление разметки в реплики с помощью библиотеки silero

Сопоставление ожидаемых результатов с полученными для *baseline* модели показало, что качество выделения сущностей неудовлетворительное.

Так как одним из признаков выделения именованных сущностей является капитализация первого символа в слове, а исследуемый датасет получен на основе автоматического преобразования речи в текст типы `speech-to-text` и по этой причине лишён данного признака, то появилась гипотеза, что разметка текста путём замены букв на заглавные и дополнительная расстановка знаков препинания сможет улучшить решение поставленной `NER` задачи.

Для добавление разметки была использована библиотека `silero`.

In [16]:
from silero import silero_stt, silero_tts, silero_te

In [17]:
# загрузка модели
model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')

Using cache found in C:\Users\Oleg/.cache\torch\hub\snakers4_silero-models_master


In [18]:
# рекапитализация и репунктуация исходных текстов
punc = []
for t in df['text'].values:
    try:
        punc.append(apply_te(t.lower(), lan='ru'))
    except:
        print(t)
        punc.append(t)
df['re_text'] = punc

# оценка полученного результата
df.head(5)

+
+ месяц 2 3 все что угодно


,dlg_id,line_n,role,text,event,company,person,person_mod,company_mod,re_text
0,0,0,manager,Алло,0.0,0.0,0.0,0.0,0.0,Алло
1,0,1,client,Алло здравствуйте,1.0,0.0,0.0,0.0,0.0,Алло здравствуйте!
2,0,2,manager,Добрый день,1.0,0.0,0.0,0.0,0.0,Добрый день!
3,0,3,client,Меня зовут ангелина компания диджитал бизнес з...,0.0,1.0,1.0,1.0,0.0,"Меня зовут Ангелина, компания диджитал бизнес,..."
4,0,4,manager,Ага,0.0,0.0,0.0,0.0,0.0,Ага


In [19]:
# обнуление показателей работы модели
df[['company_mod', 'person_mod']] = 0

Проверка того, насколько разметка текста повлияла на качество выделения сущностей.

In [20]:
# получение именовынных сущностей после обработки исходных текстов
line_n = []
for i, s in enumerate(df['re_text'].values):
    doc = nlp(s)
    if doc.ents:
        for ent in doc.ents:
            print(i, ent.text, ent.label_)
            if ent.label_=='EVT':
                df.loc[i, 'event_mod'] = 1
            elif ent.label_=='ORG':
                df.loc[i, 'company_mod'] = 1
            elif ent.label_=='PER':
                df.loc[i, 'person_mod'] = 1
            line_n.append(i)
            
df[['company_mod', 'person_mod']] = df[['company_mod', 'person_mod']].fillna(0)

3 Ангелина PER
111 Ангелина PER
137 Админа PER
158 Ру PER
159 Инфо Денис Дэ е эн PER
167 Ангелина PER
175 20.% ORG
196 Ватсапа PER
205 Да LOC
213 Кэшбэка PER
229 астана LOC
229 Кз PER
232 Жесупов Собачка PER
250 Алло Дмитрий PER
251 Максим PER
251 Китобизнес PER
253 Дмитрий PER
253 Анастасия PER
259 Сэйл PER
264 Мессенджер PER
268 Может PER
282 амо ORG
282 Аналитика PER
285 Си Эр Эм PER
319 айдар PER
338 Анастасия PER
348 Ла ла ла PER
351 Эксель LOC
358 Анастасия PER
362 Тарифа PER
410 амо ORG
424 Выходит PER
426 Блин PER
438 Вячеслав PER
444 максим PER
449 Я PER
472 Дмитрий PER


In [21]:
# вычисление метрик для модели с добавлением разметки
df_res = pd.DataFrame([f1_score(df['person'], df['person_mod']), 
                       f1_score(df['company'], df['company_mod']), 0],
             index=['PER', 'ORG', 'MEAN'], columns=['F1'])
df_res.loc['MEAN', 'F1'] = df_res.iloc[:2, 0].mean()
df_res

,F1
PER,0.585366
ORG,0.000000
MEAN,0.292683


> * Попытка усовершенствования модели методом добавления разметки в реплики ухудшило среднее значение по *F1*-мере до величины `0.29`, так как вместе с улучшением выделения именованных сущностей также добавилось существенное количество ложноположительных (`False Positive`) результатов.
> * При этом данная модель также не справляется с задачей выделения сущностей в виде названий компаний (`ORG`) и событий в виде приветствий и прощаний (`EVT`).

### Разметка исходных данных и дообучение модели spaCy

Для устранения данного недостатка было проведено её дообучение на размеченных данных из файла `text_labels_without_indexes.csv`. После этого дообученная модель использована для решения задачи.

In [22]:
# загрузка дообученной модели
nlp = spacy.load('./Documents/NER2/pipeline2')

In [23]:
# получение именованных сущностей после дообучения модели
line_n = []
for i, s in enumerate(df['re_text'].values):
    doc = nlp(s)
    if doc.ents:
        for ent in doc.ents:
            print(i, ent.text, ent.label_)
            if ent.label_=='EVENT':
                df.loc[i, 'event_mod'] = 1
            elif ent.label_=='COMPANY':
                df.loc[i, 'company_mod'] = 1
            elif ent.label_=='PERSON':
                df.loc[i, 'person_mod'] = 1
            line_n.append(i)

df[['company_mod', 'person_mod', 'event_mod']] = df[['company_mod', 'person_mod', 'event_mod']].fillna(0)

1 здравствуйте EVENT
2 Добрый день EVENT
3 Ангелина PERSON
3 диджитал бизнес COMPANY
108 до свидания EVENT
109 здравствуйте EVENT
110 здравствуйте EVENT
111 Ангелина PERSON
111 диджитал бизнес COMPANY
159 Денис Дэ EVENT
162 всего хорошего EVENT
163 До свидания EVENT
165 Здравствуйте EVENT
166 здравствуйте EVENT
167 Ангелина PERSON
167 диджитал бизнес COMPANY
177 Диджитал бизнес COMPANY
249 Добрый день EVENT
250 Дмитрий PERSON
250 добрый день EVENT
251 Максим PERSON
251 Китобизнес PERSON
253 Дмитрий PERSON
253 Анастасия PERSON
300 всего доброго EVENT
301 до свидания EVENT
304 Добрый EVENT
335 до свидания EVENT
336 до свидания EVENT
338 Анастасия PERSON
358 здравствуйте EVENT
358 Анастасия PERSON
438 Вячеслав PERSON
444 максим PERSON
472 Дмитрий PERSON
479 до свидания EVENT


In [24]:
# вычисление метрик для дообученной модели spaCy
df_res = pd.DataFrame([f1_score(df['person'], df['person_mod']), 
                       f1_score(df['company'], df['company_mod']), 
                       f1_score(df['event'], df['event_mod']), 0],
             index=['PER', 'ORG', 'EVT', 'MEAN'], columns=['F1'])
df_res.loc['MEAN', 'F1'] = df_res.iloc[:3, 0].mean()
df_res

,F1
PER,0.585366
ORG,0.666667
EVT,0.972973
MEAN,0.741668


> * Использование размеченных данных для дообучения модели `spaCy` позволило достичь среднего значения *F1*-меры, равного `0.74`.
> * Лучше всего данная модель справилась с выделением событий в виде приветствий и прощаний (`EVT`) со значением метрики *F1*, равным `0.97`. Несколько хуже модель выделила имена (`PER`) и компании (`ORG`) для которых значений метрики *F1* составило соотвественно `0.59` и `0.67`.
> * Так как данных мало, и выделить тестовую часть из них не представлялось возможным, то проверка работы дообученной модели осществлялась на тех же самых данных, на которых проводилось обучение. Это снижает достоверность полученных результатов. С целью снижения влияния данного фактора проверка работы модели было проведена на видоизменённых путём добавления разметки данных (п.2.3).

### Оформление результатов в соотвествии с поставленной задачей

In [25]:
# получение id-шников диалогов
dialog_nums = df['dlg_id'].unique()

In [26]:
# основная функция, выполняющая поставленную задачу
def main_func(mode='hello'):
    """
    input:        mode (str) - режим работы функции: `hello`   - реплики с приветствием, 
                                                     `intro`   - реплики где менеджер представил себя,
                                                     `name`    - имя менеджера,
                                                     `company` - название компании,
                                                     `bye`     - реплики с прощанием
                                                     'he_bye'  - менеджер поздоровался и попрощался
    output: None
    description:  Функция извлекает из реплик различные данные
    """
    
    df_res = pd.DataFrame([], index=[0,1,2,3,4,5], columns=['dlg_id', 'line_n', 'text_name_info'])
    
    # основной цикл перебора диалогов по их id-шникам
    for n in dialog_nums:
        df_res.loc[n, 'dlg_id'] = n
        if mode=='hello' or mode=='intro' or mode=='name' or mode=='bye':
            df_dialog = df.loc[(df['dlg_id'] == n) & (df['role'] == 'manager')]
        elif mode=='company':
            df_dialog = df.loc[df['dlg_id'] == n]
        elif mode=='he_bye':
            cnt = df.loc[(df['dlg_id'] == n)].query('hello == 1 | bye == 1').shape[0]
            if cnt == 2:
                df_res.loc[n, 'text_name_info'] = 'В данном диалоге менеджер поздоровался и попрощался'
            df_dialog = df.loc[df['dlg_id'] == n]
        
        indexes = df_dialog.index
        
        # перебор реплик внутри диалога
        for i, s in enumerate(df_dialog['text'].values):
            doc = nlp(s)
            for ent in doc.ents:
                if mode=='hello':
                    if 'добр' in (ent.text.lower()) or 'здрав' in (ent.text.lower()):
                        df.loc[indexes[i], 'hello'] = 1
                        df_res.loc[n, 'line_n'] =  indexes[i]
                        df_res.loc[n, 'text_name_info'] =  ent.text
                elif mode=='intro':
                    if ent.label_ == 'PERSON':
                        df_res.loc[n, 'line_n'] =  indexes[i]
                        df_res.loc[n, 'text_name_info'] = f'{s}'
                elif mode=='name':
                    if ent.label_ == 'PERSON':
                        df_res.loc[n, 'line_n'] =  indexes[i]
                        df_res.loc[n, 'text_name_info'] = f'{ent.text.title()}'
                elif mode=='company':
                    if ent.label_ == 'COMPANY':
                        df_res.loc[n, 'line_n'] =  indexes[i]
                        df_res.loc[n, 'text_name_info'] = ent.text
                elif mode=='bye':
                    if 'до свид' in (ent.text.lower()) or 'всего' in (ent.text.lower()):
                        df.loc[indexes[i], 'bye'] = 1
                        df_res.loc[n, 'line_n'] =  indexes[i]
                        df_res.loc[n, 'text_name_info'] = s
                        
    display(df_res)

In [27]:
# a. получение реплик, где менеджер поздоровался
main_func()

,dlg_id,line_n,text_name_info
0,0,2,Добрый день
1,1,109,здравствуйте
2,2,165,Здравствуйте
3,3,249,Добрый день
4,4,304,Добрый
5,5,358,здравствуйте


In [28]:
# b. получение реплик, в которых менеджер представил себя
main_func(mode='intro')

,dlg_id,line_n,text_name_info
0,0,NaN,NaN
1,1,159,Самое только вместо инфо денис дэ е эн
2,2,NaN,NaN
3,3,NaN,NaN
4,4,NaN,NaN
5,5,358,Да да да здравствуйте анастасия меня слышно да...


In [29]:
# c. извлечение имени менеджера из реплик
main_func(mode='name')

,dlg_id,line_n,text_name_info
0,0,NaN,NaN
1,1,159,Денис
2,2,NaN,NaN
3,3,NaN,NaN
4,4,NaN,NaN
5,5,358,Анастасия


In [30]:
# d. извлечение названий компаний
main_func(mode='company')

,dlg_id,line_n,text_name_info
0,0,3,диджитал бизнес
1,1,111,диджитал бизнес
2,2,177,Диджитал бизнес
3,3,NaN,NaN
4,4,NaN,NaN
5,5,NaN,NaN


In [31]:
# e. получение реплик, где менеджер попрощался
main_func(mode='bye')

,dlg_id,line_n,text_name_info
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,301,Да до свидания
4,4,336,Угу да до свидания
5,5,NaN,NaN


In [32]:
# f. проверка требования к менеджеру о наличии приветствия и прощания
main_func(mode='he_bye')

,dlg_id,line_n,text_name_info
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,NaN,В данном диалоге менеджер поздоровался и попро...
4,4,NaN,В данном диалоге менеджер поздоровался и попро...
5,5,NaN,NaN


### Выводы по использованным моделям

> * Для решения задачи использованы три подхода на основе библиотеки `spaCy`.
> * В качестве *baseline* решения выступила предобученная модель `ru_core_news_lg` для библиотеки `spaCy`, которая позволила достичь среднего значения по *F1*-мере, равного `0.38`.
> * Для улучшения *baseline* результата проверена гипотеза о том, что на качество работы модели может влиять отсутствие разметки в репликах. Для добавления разметки была использована библиотека `silero`. Добавление разметки ухудшило среднее значение *F1*-меры, которое снизилась до `0.29`. Ухудшение вызвано появлением большого количества ложноположительных (`False Positive`) срабатываний.
> * И первая и вторая модели справлялись только с выделением именованных сущностей в виде имён (`PER`), но не могли выделить из реплик другие типы сущностей, такие как сущности в виде названий компаний (`ORG`) и событий в виде приветствий и прощаний (`EVT`).
> * В качестве третьей была использована дообученая модель `ru_core_news_lg`. Данный подход обеспечил получение среднего значения *F1*-меры, равного `0.74`. Лучше всего данная модель справилась с выделением событий в виде приветствий и прощаний (`EVT`) со значением метрики *F1*, равным `0.97`. Несколько хуже модель выделила имена (`PER`) и компании (`ORG`) для которых значений метрики *F1* составило соотвественно `0.59` и `0.67`.
> * Недостатком третьей модели стало использование для обучения и тестирования одни и тех же данных из-за того, что их объём не позволил выделить отдельные обучающую и тестовую части. Для минимизации влияния данного фактора проверка тестирование проводилось на видоизменённых (капитализированных и пунктуализированных) данных.
> * Результаты работы лучшей (третьей) модели оформлены в соотвествии с поставленной задачей.

## Общие выводы

> 1. Полученные в файлe `test_data.csv` исходные данные с диалогами загружены и проанализированы:
>    * в процессе анализа полученных данных установлено, что они содержат `480` реплик, образующих `6` диалогов между менеджером и клиентами;
>    * все реплики получены на основе автоматического распознавания речи и преобразования её в тексты, то есть `STT` (`speech-to-text`) модели;
>    * решаемая задача сведена к выделению необходимых именованных сущностей (`NER`) из реплик;
>    * в качестве контрольной метрики выбрана *F1*-мера как гармоническое среднее между точностью (*precision*) и полнотой (*recall*).
>
>
> 2. Для выделения из реплик именованных сущностей использованы различные модели, и проведён их сравнительный анализ:
>    * для решения поставленной задачи определены три базовых типа именованных сущностей, которые модель должны выделять в репликах:
>       * `PERSON` - имя менеджера;
>       * `COMPANY` - название компании;
>       * `EVENT` - событие (приветствие или прощание).
>    *  чтобы иметь возможность оценки по *F1*-мере качества работы различных моделей была произведена ручная разметка полученных данных на наличие в них именованных сущностей с использованием библиотеки `spacy_annotator` результатом которой стало присвоение найденным сущностям одной из указанных выше категорий;
>    * в качестве *baseline* решения выступила предобученная модель `ru_core_news_lg` из библиотеки `spaCy`, которая позволила достичь среднего значения по *F1*-мере, равного `0.38`, при этом она успешно выделяла только один вид сущностей - имена (`PER`);
>    * для улучшения *baseline* результата осуществлена попытка добавление разметки в реплики для чего была использована библиотека `silero`, что в результате лишь ухудшило среднее значение значение *F1*-меры, которая снизилась до `0.29` за счёт появления большого количества ложноположительных (`False Positive`) срабатываний;
>    * также как и первая, вторая модели справлялась только с выделением именованных сущностей в виде имён (`PER`), но не выделяла такие сущности как названия компаний (`ORG`) и события в виде приветствий и прощаний (`EVT`);
>    * было произведено дообучение модели `ru_core_news_lg` на размеченных данных, что обеспечило получение среднего значения *F1*-меры, равного `0.74` в основном за счёт того, что данная модель лучше всего справилась с выделением событий в виде приветствий и прощаний (`EVT`) со значением метрики *F1*, равным `0.97` и несколько хуже модель выделила имена (`PER`) и компании (`ORG`) для которых значений метрики *F1* составило соотвественно `0.59` и `0.67`;
>    * недостатком третьей модели отмечено использование для обучения и тестирования одинаковых данных из-за того, что их объём не позволил выделить отдельные обучающую и тестовую части, поэтому для минимизации влияния данного фактора проверка тестирование проводилось на видоизменённых (капитализированных и пунктуализированных) данных;
>    * результаты работы лучшей по *F1*-мере (третьей) модели оформлены в соотвествии с поставленной задачей.